Здесь буду анализировать данные из разных открытых источников правительства по бюджету и подобным вещам.
Сейчас пока общие вопросы:
1. Анализ бюджетов текущего и прошлых годов (как менялся объём и распределение)
2. Источники пополнения бюджета
3. Доходы и расходы по регионам (какие регионы приносят больше всего доходов, какие меньше, как абсолютно так и на душу населения)
4. Параметры рождаемости и смертности
5. Инфляция
6. Данные о зарплатах по стране и регионам

## Источники данных
### Федеральная служба гос статистики (http://www.gks.ru/)

Содержит огромное количество данных, выпускаются различные итоговые документы за год, например:

Финансы России 2018 (http://www.gks.ru/free_doc/doc_2018/fin18.pdf)

### БД (http://cbsd.gks.ru/#)
Удобный инструмент просмотра статистики на сайте с возможность построения графиков


## Open data (http://www.gks.ru/opendata/dataset)
Непонятно что там есть

## Регионы России

http://www.gks.ru/free_doc/new_site/region_stat/sep_region.html

Здесь можно скачать справочник "Регионы России. Социально-экономические показатели", в котором много всего интересного

### Экономические показатели субъектов
Попробуем из pdf выше скопировать текст в текстовый файл и затем из этого файла сделать csv. Нужно учесть, что таблица разделена на 2 страницы, поэтому удобнее будет сделать 2 csv, один для одного набора колонок, другой для другого. На втором этапе нам нужно вытащить из этого списка федеральные округа при чтении. После чтения 2-х csv и фильтрации по округам, смёржим оба pandas объекта в один. Итак скопируем текст по первому набору колонок и создадим файл.

Тонкости:

Есть разные суммарные цифры и "в том числе", например "Архангельская область", в том числе "Ненецкий автономный округ", Архангельская область без автономного округа". Есть переносы строк в названии, нужно это всё править руками. Руами же удобнее выделить в отдельный документ федеральные округа. Хотя с другой стороны лучше попытаться всё автоматизировать, чтобы была возможность использовать этот парсер для даных за другие года.

Ещё местами к данным прикреплена скобка с цифрой и перед ней может быть ...

Есть ещё вот такие казусы: "24284,910140135,8 3643400", когда 2 числа слиплись, поэтому нужно в конце проверить, что во всех строках число колонок одинаковое и правильное.

У Питера вообще нет 3-х колонок во второй таблице!
Тут придётся тупо руками править(

In [2]:
import os
import os.path
import io
import pandas as pd

core_path = os.environ['GITHUB_PATH']
file_path1 = os.path.join(core_path, 'education/Statistics/Rosstat/reg_economy1.txt')
file_path2 = os.path.join(core_path, 'education/Statistics/Rosstat/reg_economy2.txt')

from locale import *
setlocale(LC_NUMERIC, '')

def check_columns_count(samples, num):
    for i, s in enumerate(samples):
        if len(s) != num:
            return i
    return -1

def num_there(s):
    return any(i.isdigit() for i in s)

def filter_parts(parts, is_name_first=True):
    real_parts = []
    last_is_num = True
    for i, p in enumerate(parts):
        p = p.strip(' \t\n\r')
        if not p:
            continue
            
        if not is_name_first and any(i.isalpha() for i in p):
            continue
            
        if p.endswith(')'):
            p = p[0:-2]
        
        if p == '...':
            p = '0'

        if num_there(p):
            val = float(p.replace(',', '.'))
            real_parts.append(val)
            last_is_num = True
        else:
            if i == 0 or last_is_num:
                real_parts.append(p)
            else:
                real_parts[-1] = real_parts[-1] + ' ' + p
            last_is_num = False
    return real_parts

def parse_datafile(file_path, is_name_first=True):
    lines = []
    with io.open(file_path, mode="r", encoding="utf-8") as f:
        curr_str = ''
        for l in f:
            if ':' in l:
                continue
                 
            s = l
            if curr_str:
                s = curr_str + ' ' + l
                curr_str = ''
                
            parts = s.split(' ')
            if len(parts) < 5: # part of name
                if is_name_first:
                    curr_str = s.strip(' \t\n\r')
                continue
            
            real_parts = filter_parts(parts, is_name_first)
            lines.append(real_parts)
            
    return lines

def create_df():
    samples1 = parse_datafile(file_path1)
    err_line_num = check_columns_count(samples1, 10) 
    if err_line_num != -1:
        print('Part 1. Invalid number of columns in line ', err_line_num)
        print(samples1[err_line_num])

    samples2 = parse_datafile(file_path2, False)

    # Fix Piter
    samples2[32] = [samples2[32][0], samples2[32][1], samples2[32][2], samples2[32][3], 0, 0, 0, samples2[32][4], samples2[32][5], samples2[32][6]]

    err_line_num = check_columns_count(samples2, 10) 
    if err_line_num != -1:
        print('Part 2. Invalid number of columns in line ', err_line_num)
        print(samples2[err_line_num])

    headers1 = ['Name', 'Square', 'Population', 'Workers', 'MeanMonthIncome', 'MeanMonthCost', 'MeanMonthSalary', 'GDP', 'CapitalInvestments', 'Funds']
    df1 = pd.DataFrame(samples1, columns=headers1)

    headers2 = ['MineralsVolume', 'ManufactoreVolume', 'EnergyVolume', 'WaterVolume', 'AgricultureVolume', 'VegetableVolume', 'AnimalVolume', 'NewBuildingSquare', 'RetailOverturn', 'FinResult']
    df2 = pd.DataFrame(samples2, columns=headers2)

    df = df1.copy()
    df[df2.columns] = df2[df2.columns]
#     df.head()
    df_subjects = df[~((df.Name.str.contains('округ') | df.Name.str.contains('Федерация')))]
    subjects_csv = os.path.join(core_path, 'education/Statistics/Rosstat/subjects.csv')
    df_subjects.to_csv(subjects_csv, sep='\t', encoding='utf-8')

In [3]:
# Uncomment to create csv from text data files
# create_df()

## Reading

In [7]:
file_path = os.path.join(core_path, 'education/Statistics/Rosstat/subjects.csv')
df_subjects = pd.read_csv(file_path, sep='\t', encoding='utf-8')

In [8]:
df_subjects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 21 columns):
Unnamed: 0            81 non-null int64
Name                  81 non-null object
Square                81 non-null float64
Population            81 non-null float64
Workers               81 non-null float64
MeanMonthIncome       81 non-null float64
MeanMonthCost         81 non-null float64
MeanMonthSalary       81 non-null float64
GDP                   81 non-null float64
CapitalInvestments    81 non-null float64
Funds                 81 non-null float64
MineralsVolume        81 non-null object
ManufactoreVolume     81 non-null float64
EnergyVolume          81 non-null float64
WaterVolume           81 non-null float64
AgricultureVolume     81 non-null float64
VegetableVolume       81 non-null float64
AnimalVolume          81 non-null float64
NewBuildingSquare     81 non-null float64
RetailOverturn        81 non-null float64
FinResult             81 non-null float64
dtypes: float64(18)

In [10]:
# df_subjects['MineralsVolume']
df_subjects.sort_values('GDP')

,Unnamed: 0,Name,Square,Population,Workers,MeanMonthIncome,MeanMonthCost,MeanMonthSalary,GDP,CapitalInvestments,...,MineralsVolume,ManufactoreVolume,EnergyVolume,WaterVolume,AgricultureVolume,VegetableVolume,AnimalVolume,NewBuildingSquare,RetailOverturn,FinResult
61,74,Республика Алтай,92.9,218.1,83.4,18411.0,11813.0,26316.0,46128.4,13111.0,...,3593.0,4995.0,2822.0,356.0,10932.1,2199.4,8732.8,130.9,24371.4,19875.0
80,94,Еврейская автономная область,36.3,162.0,67.2,23386.0,16574.0,34409.0,46872.2,10455.0,...,7593.0,6743.0,4209.0,446.0,5216.3,4273.2,943.1,50.4,22582.5,64.0
37,44,Республика Ингушетия,3.6,488.0,179.4,15131.0,7283.0,22750.0,50882.9,13587.0,...,1463.0,1865.0,1345.0,487.0,9031.3,1952.7,7078.7,309.3,22500.8,-1465.0
63,76,Республика Тыва,168.6,321.7,98.7,14048.0,7657.0,31251.0,52221.3,9342.0,...,23347.0,370.0,4669.0,326.0,5710.0,904.9,4805.2,101.0,22139.9,6732.0
29,35,Республика Калмыкия,74.7,275.4,111.1,14730.0,7940.0,22919.0,56045.1,10448.0,...,…,1235.0,2054.0,395.0,27084.3,4974.6,22109.7,96.3,19631.9,745.0
35,41,г. Севастополь,0.9,436.7,182.1,24713.0,20666.0,27687.0,64163.2,30382.0,...,…,13310.0,6492.0,1729.0,2474.3,2279.1,195.2,174.3,63290.1,620.0
39,46,Карачаево-Черкесская Республика,14.3,466.3,169.2,17142.0,9429.0,22638.0,73151.3,17812.0,...,2751.0,35194.0,9061.0,991.0,27468.6,12924.8,14543.8,201.8,37075.6,904.0
28,34,Республика Адыгея,7.8,453.4,152.1,24677.0,19245.0,24490.0,91352.4,22858.0,...,2740.0,47823.0,2346.0,1807.0,21612.8,12310.1,9302.7,244.3,86249.4,405.0
40,47,Республика Северная Осетия - Алания,8.0,701.8,289.7,22773.0,17039.0,24715.0,125498.3,26807.0,...,661.0,27588.0,6697.0,1542.0,21926.6,9581.7,12344.9,192.2,109722.5,-362.0
38,45,Кабардино-Балкарская Республика,12.5,865.8,362.6,20385.0,15423.0,22782.0,132706.9,40875.0,...,398.0,27688.0,8733.0,1069.0,45570.2,24745.7,20824.6,426.8,123965.1,-801.0


In [22]:
df_subjects['gdp_per_capita'] = df_subjects['GDP']*1000/(df_subjects['Population'])
df_subjects['gdp_per_worker'] = df_subjects['GDP']*1000/(df_subjects['Workers'])
df_subjects[['Name', 'gdp_per_capita', 'gdp_per_worker', 'MeanMonthSalary']].sort_values('gdp_per_capita')

,Name,gdp_per_capita,gdp_per_worker,MeanMonthSalary
37,Республика Ингушетия,1.042682e+05,2.836282e+05,22750.0
41,Чеченская Республика,1.160134e+05,3.246567e+05,23249.0
35,г. Севастополь,1.469274e+05,3.523515e+05,27687.0
38,Кабардино-Балкарская Республика,1.532766e+05,3.659870e+05,22782.0
39,Карачаево-Черкесская Республика,1.568760e+05,4.323363e+05,22638.0
63,Республика Тыва,1.623292e+05,5.290912e+05,31251.0
30,Республика Крым,1.650826e+05,3.759144e+05,26165.0
4,Ивановская область,1.770485e+05,3.936739e+05,23470.0
40,Республика Северная Осетия - Алания,1.788235e+05,4.332009e+05,24715.0
36,Республика Дагестан,1.948813e+05,5.470423e+05,21941.0


## Платёжный баланс

# Демография

## Естественный прирост

Естественный прирост не учитывает миграцию, только рождаемость и смертность, для независимости от общей численности населения страны, удобно брать показатели на 1000 человек

Возьмём таблицу "
http://www.gks.ru/wps/wcm/connect/rosstat_main/rosstat/ru/statistics/population/demography/#